In [2]:
import copy
def getDegreeMatrix(graph):
    degreeMatrix = np.zeros((len(graph.nodes()),len(graph.nodes())))
    for i in range(len(graph.nodes())):
        degreeMatrix[i][i] = graph.degree(graph.nodes()[i])
    return degreeMatrix

def getNodeToNodeProbMatrix(G):
    D = getDegreeMatrix(G)
    A = nx.adjacency_matrix(G).toarray()
    P = np.dot(np.add(np.identity(len(G.nodes())), A), np.linalg.inv(np.add(np.identity(len(G.nodes())), D)  ))
    return P

def explore(probability_matrix, i, T=100, r = 2, e=.00001, epsilon=.00001):
    x = np.zeros(len(probability_matrix))
    x[i] = 1
    x_prev = copy.deepcopy(x)
    for t in range(T):
        x = np.dot(probability_matrix, x)
        x = (x * (x > e))**r
        x = x/sum(x)
        if max(x - x_prev) < epsilon:
            return x_prev
        x_prev = copy.deepcopy(x)
    return x

def merge(G, treshold):
    D = dict()
    P = getNodeToNodeProbMatrix(G)
    for index in range(len(G.nodes())):
        F = set()
        S = set()
        x = explore(P, index)
        m = np.argmax(x)
        S.add(index)
        for j in range(len(x)):
            if x[j] > treshold * x[m]:
                F.add(j)
        if m not in D.keys():
            D[m] = [S, F]
        else:
            D[m][0].update(S)
            D[m][1].update(F)
    
    passed = dict()
    dkeys1 = list(D.keys())
    for m1 in dkeys1:
        if m1 in D.keys(): #if m1 still in keys
            dkeys2 = list(D.keys())
            for m2 in dkeys2:
                if m2 in D.keys() and m2 != m1: #if m1 still in keys
                    if m1 in passed.keys():
                        if m2 in passed[m1]:
                            break;
                    elif m2 in passed.keys():
                        if m1 in passed[m2]:
                            break;
                    if m1 not in passed.keys():
                        passed[m1] = set()
                    passed[m1].add(m2)
                    if len(D[m1][1] & D[m2][1]) > min(len(D[m1][1]),len(D[m2][1]))/2:
                        D[m1][0].update(D[m2][0])
                        D[m1][1].update(D[m2][1])
                        del D[m2]
    return D

def get_name(person):
    name = 'unknown'
    with urllib.request.urlopen("https://api.vk.com/method/users.get?user_id="+str(person)+"&v=5.52") as url:
            received = json.loads(url.read().decode())
            if 'error' not in received:
                name = str(received['response'][0]['first_name'])+' '+str(received['response'][0]['last_name'])
    return name

In [3]:
import networkx as nx
import urllib.request, json
import numpy as np
import pandas as pd

user_id = 2598011
print("User: " + get_name(user_id))

G=nx.Graph()
with urllib.request.urlopen("https://api.vk.com/method/friends.get?user_id="+str(user_id)+"&v=5.52") as url:
    G.add_nodes_from(json.loads(url.read().decode())['response']['items'])

for node in G.nodes():
    with urllib.request.urlopen("https://api.vk.com/method/friends.get?user_id="+str(node)+"&v=5.52") as url:
        received = json.loads(url.read().decode())
        if 'error' not in received:
            for friend in received['response']['items']:
                if friend in G.nodes():
                    G.add_edge(node, friend)

print("Nodes: " + str(len(G.nodes())))
print("Edges: " + str(len(G.edges())))


graphs = list(nx.connected_component_subgraphs(G))
for graph in graphs:
    print("Connected graph " + str(graphs.index(graph)))
    communities = merge(graph, 0.5)
    keys = list(communities.keys())
    for key_ind in range(len(keys)):
        print("Community " + str(key_ind + 1) + ":")
        for person_index in communities[keys[key_ind]][0]:
            print("      " + str(graph.nodes()[person_index])+ ": " + get_name(graph.nodes()[person_index]))
        print('\n')
    print("\n\n")

User: Юрий Дудь
Nodes: 379
Edges: 2100
Connected graph 0
Community 1:
      557957: Екатерина Данилова
      2055928: Оля Лукьянова
      966822: Павел Власов-Мрдуляш


Community 2:
      262400: Гульнара Хаматова
      5544519: Даниил Иванов
      7858446: Александр Кунда
      17867151: Сергей Кривохарченко


Community 3:
      6145024: Иван Макаров
      4396635: Евгений Зырянкин
      1025027: Дмитрий Навоша
      4299: Egor Kretsan
      1287489: Дарья Иванова
      3415537: Сергей Даровских
      2173961: Андрей Шумаков
      444626: Антон Матвеев
      729486: Роман Мун
      294412: Стас Володин
      140622861: Ольга Байцур
      4101467: Сергей Гилёв
      1564856: Алексей Пономарев
      37567794: Алексей Серяков
      23823580: Артур Плотников
      1137557: Виктор Орлов
      1723712: Станислав Рынкевич
      825369: Михаил Хомич
      53947196: Александр Ткач
      49369116: Станислав Чудин
      433479: Александр Скворцов
      1077147: Михаил Калашников
      17750566: 